<a href="https://colab.research.google.com/github/mahatigorthy/FutureMakers2022/blob/main/mahati_is_trying_this_for_the_sixth_time_now_pray_for_her.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
##import libraries

import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
import requests
import time
import numpy as np
import pandas as pd

In [39]:
nRowsRead = 84502
df1 = pd.read_csv('/content/ExtractedTweets.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'tweets.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 84502 rows and 3 columns


In [40]:
df1.head(84502)

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...
...,...,...,...
84497,Republican,davereichert,Great meeting with @PSEClassified to discuss s...
84498,Republican,davereichert,Congratulations James Paxton on throwing a no-...
84499,Republican,davereichert,RT @WAWheat23: @davereichert - Thank you for y...
84500,Republican,davereichert,See my statement on the Iran nuclear deal here...


In [41]:
#transforming the target into 1 for Democrats and 0 for Conservative
df1['Party'] = df1['Party'].map(lambda x : 1 if x == 'Democrat' else 0)

In [42]:
df1.head(84502)

,Party,Handle,Tweet
0,1,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,1,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,1,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,1,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,1,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...
...,...,...,...
84497,0,davereichert,Great meeting with @PSEClassified to discuss s...
84498,0,davereichert,Congratulations James Paxton on throwing a no-...
84499,0,davereichert,RT @WAWheat23: @davereichert - Thank you for y...
84500,0,davereichert,See my statement on the Iran nuclear deal here...


In [43]:
# import text based modules
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction import _stop_words


In [44]:
additional_politics_english_stop = ['www', 'things', 'does', 'x200b', 'amp', 'want', 'watch',
                           'just', 'like', 'https', 'com', 'republican', 'republicans',
                           'libertarians', 'democrats', 'democrat', 'people', 'libertarian',
                           'says', 'say', 'did', 'this', 'conservative', 'conservatives' ]

additional_english_stop = ['www', 'things', 'does', 'x200b', 'amp',
                           'just', 'like', 'https', 'com', 'watch', 'want',
                           'says', 'say', 'did', 'this']

new_stop_list = _stop_words.ENGLISH_STOP_WORDS.union(additional_english_stop)
new_politics_english_stop_list = _stop_words.ENGLISH_STOP_WORDS.union(additional_politics_english_stop)
print(len(_stop_words.ENGLISH_STOP_WORDS))
print(len(additional_english_stop))
print(len(new_politics_english_stop_list))
print(len(new_stop_list))

318
15
341
332


In [45]:
# code originally written by boom
# Function to count words in each df
def word_counter(title_df,stop_list = []):
    
    # Count Vectorize
    cvec = CountVectorizer(stop_words = stop_list, max_features=15)

    # Transform the corpus
    X_text = cvec.fit_transform(title_df['Tweet'])

    # Converts text to array form
    X_text = pd.DataFrame(X_text.toarray(), columns= cvec.get_feature_names())

    # See word counts
    word_counts = X_text.sum().sort_values(0, ascending=False)
    
    return word_counts

In [46]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score, confusion_matrix

In [47]:
df1.isnull().sum()


Party     0
Handle    0
Tweet     0
dtype: int64

In [48]:
X = df1[['Tweet', 'Handle']]
y = df1['Party']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size = 0.25)

In [50]:
y_test.value_counts(normalize=True)


1    0.502887
0    0.497113
Name: Party, dtype: float64

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier


i tried AdaBoost with TFIDF

In [37]:
ada_pipe = Pipeline([
        ('tvec', TfidfVectorizer()),
        ('ada', AdaBoostClassifier())
])

ada_params = {
    'tvec__max_features': [None,500,1000],
    'tvec__min_df': [2,3,4],
    'tvec__max_df': [.5,.4,.3],
    'tvec__ngram_range': [(1,1),(1,3)],
    'tvec__stop_words': [None, 'english', new_stop_list],
    'ada__learning_rate': [.5]}

gs= GridSearchCV(ada_pipe, 
                   param_grid=ada_params, 
                   cv = 3,
                   verbose = 1,
                   n_jobs = -1)

gs.fit(X_train['Tweet'],y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tvec', TfidfVectorizer()),
                                       ('ada', AdaBoostClassifier())]),
             n_jobs=-1,
             param_grid={'ada__learning_rate': [0.5],
                         'tvec__max_df': [0.5, 0.4, 0.3],
                         'tvec__max_features': [None, 500, 1000],
                         'tvec__min_df': [2, 3, 4],
                         'tvec__ngram_range': [(1, 1), (1, 3)],
                         'tvec__stop_words': [None, 'english',
                                              frozenset({'a', 'about', 'above',
                                                         'across', 'after',
                                                         'afterwards', 'again',
                                                         'against', 'all',
                                                         'almost', 'alone',
                                                         'along', 'alread

In [53]:
tfada_bestscore = gs.best_score_
tfada_params = gs.best_params_
tfada_train = gs.score(X_train["Tweet"],y_train)
tfada_test= gs.score(X_test["Tweet"],y_test)
tfada = ('TF-IDF with AdaBoost',tfada_bestscore, tfada_params, tfada_train, tfada_test)

In [55]:
print(f'Best Score: {gs.best_score_}')
print(f'Best Parameters: {gs.best_params_}')
print(f'Train Accuracy Score: {gs.score(X_train["Tweet"],y_train)}')
print(f'Test Accuracy Score: {gs.score(X_test["Tweet"],y_test)}')

Best Score: 0.6063020587847014
Best Parameters: {'ada__learning_rate': 0.5, 'tvec__max_df': 0.3, 'tvec__max_features': 500, 'tvec__min_df': 4, 'tvec__ngram_range': (1, 1), 'tvec__stop_words': None}
Train Accuracy Score: 0.6060969452158546
Test Accuracy Score: 0.6068825144371864
